# between and within ratio (組間和組內比)

<font size = 5 color = orange>此比較用於單個特徵, 去比較不同 label(y值) 對應的數值.</font>

期望來說, 相同 labe 所對應的數值差異越小越好; 不同的 label 所對應的數值差異越大越好.

比較方法是使用 ANOVA, 做法和 One-way ANOVA 一樣(參考 : 分析相關/015.假設檢定/03.ANOVA.ipynb). 但是我們要的是"有差異", 所以假設檢定得到的 p_value 越小越好. 主要是因為 $H_0$ 基本上是設 : 平均數相等.



![參考圖片](https://imgur.com/QLUorPT.jpg)

## One-way ANOVA table

<font size = 5>符號 : </font>

*   k : 組數

*   N : 總樣本數 ( $\sum n_i$ )

*   $y_{k,i}$ : 第k組第i個觀察值 (上面表示成 $x_{i,k}$)

*   $\bar{y_{k}}$ : 第k組觀察值的平均值 (上面表示成 $\bar{x_{k}}$)

*   $\bar{y}$    : 所有觀察值的平均值 (上面表示成 $\bar{\bar{x}}$)

*   $SS_T (總變異) : \sum\limits_{k}\sum\limits_{i}(y_{k,i} - \bar{y})^2 ~=~ n_T\sigma_T^2 ~=~ (n_T-1)s_T^2 $

*   $SS_G (組間變異, SS_A (A 因子的變異)) : \sum\limits_{k}n_k(\bar{y_{k}} - \bar{y})^2 = SS_T - SS_W = SS_T - SS_E$

*   $SS_W (組內變異, SS_E) : \sum\limits_{k}\sum\limits_{i}(y_{k,i} - \bar{y_{k}})^2  = \sum\limits_{k}(n_k-1)(\frac{1}{n_k-1}\sum\limits_{i}(y_{k,i} - \bar{y_{k}})^2) = \sum\limits_{k} (n_k-1)s_k^2$

*   $SS_T = SS_G + SS_W$

*   $F^*$   : 若 $F^* > F_\alpha(k-1,n-k)$，則 $ reject ~~H_0 $ 。


<img src="https://i.imgur.com/R2OoUE9.png" width="60%" height="60%">

### python

In [1]:
from pickle import FALSE
import numpy as np
import pandas as pd

##input
G = 12 #組別



##建立資料
data = {}
n = []
for i in range(G):
    ni = np.random.randint(12,30)
    data["母體{}".format(i+1)] = np.random.normal(loc = 0, scale = 1 ,size = ni)
    n.append(ni)
N = np.sum(n) #總樣本數

##處理資料(python 不同長度製作成data frame很麻煩，所以直接處理資料在製作，之後計算也比較方便)
pre_data = {"ni"   : [ len(data[i]) for i in data.keys()],
            'xbar' : [ np.mean(data[i]) for i in data.keys()] ,
            's2'  :  [ np.var(data[i],ddof = 1) for i in data.keys()]}
df = pd.DataFrame(pre_data)
df.index = data.keys()
df

##run
all_mean = df['ni'] * df['xbar']/np.sum(df['ni']) #sum(ni*xbar/sum(ni))

SSG =np.sum(df['ni']*(df['xbar']-all_mean)**2)  #sum(nk(xbar-xbarbar))
SSW = np.sum((df['ni']-1) * df['s2'] ) #sum((nk-1)*sk^2)
SST = SSG + SSW

SS = [SSG,SSW,SST]
df = [G-1,N-G,N-1]
MS = [SS[0]/df[0],SS[1]/df[1],np.NaN]
F = [ MS[0]/MS[1],np.NaN,np.NAN]

ANOVA_df = pd.DataFrame([SS,df,MS,F]).T
ANOVA_df.index = ["因子(組間)","隨機(組內)","總和"]
ANOVA_df.columns = ["平方和","自由度","平均平方和","F"]

ANOVA_df #用print 會沒對齊



,平方和,自由度,平均平方和,F
因子(組間),7.445084,11.0,0.676826,0.629811
隨機(組內),251.467837,234.0,1.074649,NaN
總和,258.912921,245.0,NaN,NaN
